In [1]:
from json import dumps,loads
import pandas as pd
from collections import defaultdict

In [2]:
BB_data = pd.read_csv("../data/Hot Stuff.csv")
BB_data.head()

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/2012-1...,12/15/2012,21,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,28.0,21,11
1,http://www.billboard.com/charts/hot-100/2012-1...,12/22/2012,14,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,21.0,14,12
2,http://www.billboard.com/charts/hot-100/2012-1...,12/29/2012,12,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,14.0,12,13
3,http://www.billboard.com/charts/hot-100/2013-0...,1/5/2013,11,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,12.0,11,14
4,http://www.billboard.com/charts/hot-100/2013-0...,1/12/2013,15,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,11.0,11,15


In [3]:
song_art = set()
for i,r in BB_data.iterrows():
    song_art.add((r['Song'],r['Performer']))

In [4]:
from bs4 import BeautifulSoup
import requests
import re
from time import sleep
from random import random
from Levenshtein import ratio

In [5]:
def clean_art_name(art):
    if "Featuring" in art:
        art = art.split("Featuring")[0].strip()
    if "Feat." in art:
        art = art.split("Feat.")[0].strip()
    if "featuring" in art:
        art = art.split("featuring")[0].strip()
    if "&" in art:
        art = art.split("&")[0].strip()
    if "and" in art:
        art = art.split("and")[0].strip()
    if "And" in art:
        art = art.split("And")[0].strip()
    if "With" in art:
        art = art.split("With")[0].strip()
    if "with" in art:
        art = art.split("with")[0].strip()
    if "(" in art:
        art = art.split("(")[0].strip()
    return art
        
def scrape(sa,debug=False):
    s_name,a_name = sa
    if debug:
        print(s_name,a_name)
    url = "https://search.azlyrics.com/search.php"
    headers = {'Cache-Control':'no-cache'}
    querystring = {"q":"{} by {}".format(s_name,a_name)}
    try:
        resp = requests.request("GET",url,headers=headers,params=querystring)
        if debug:
            print(resp.text)

        list_of_results_reg = r'''<div class="container main-page">\s*<div class="row">\s*<div class="col-xs-12 col-sm-10 col-sm-offset-1 col-md-8 col-md-offset-2 text-center" style="padding-top: 20px;">\s*<div class="panel">\s*<div class="panel-heading"><b>Song results:</b><br><small>\[[\s\S]*\]</small></div>\s*<table class="table table-condensed">\s*([\s\S]+?)\s*</table>\s*</div>'''
        m = re.search(list_of_results_reg, resp.text)
        if not m:
            if debug:
                print("cleaning artist name")
            querystring = {"q":"{} by {}".format(s_name,clean_art_name(a_name))}
            resp = requests.request("GET",url,headers=headers,params=querystring)
            m = re.search(list_of_results_reg, resp.text)
        if m:
            rows = m.group(1)
            search_data_reg = r'''<tr><td class="text-left visitedlyr">\s*\d{1,2}\. <a href="(https://www\.azlyrics\.com/lyrics/[\w-]+/[\w-]+\.html)" target="_blank"><b>([\s\S]*?)</b></a>  by <b>([\s\S]*?)</b><br/?>\s*<small>[\s\S]*?</small>\s*</td></tr>'''
            matches = re.search(search_data_reg,rows)

            soup = BeautifulSoup(rows, 'html.parser')
            result_list = []
            for r in soup.find_all('tr'):
                if debug:
                    print(str(r))
                match = re.search(search_data_reg,str(r))
                if match:
                    res = "{} by {}".format(match.group(2),match.group(3))
                    orig = querystring['q']
                    if debug:
                        print(orig,"=?=",res)
                    result_list.append((ratio(res.lower(),orig.lower()),res,match.group(1)))
                elif debug:
                    print('"{}" by "{}"'.format(s_name,a_name))
                    print("No match:\n{}".format(str(r)))
            return [sorted(result_list,reverse=True),s_name, a_name]
        else:
            return [[],s_name, a_name]
    except KeyboardInterrupt as e:
        print("My time has come")
        raise e
    except:
        print("Site thinks im tryna DDOS, lol")
        sleep(random()*20)
        return scrape(sa)

In [ ]:
scraped_data = {}
cnt = 0
song_art_list = list(song_art)
for i in range(len(song_art_list)):
    s,a = song_art_list[i]
    scraped_data[(s,a)] = scrape((s,a))
    cnt+=1
    if cnt%100 == 0:
        print("{:4.3f}% completed".format(cnt/len(song_art)*100))

In [6]:
scrape(("Don't You Worry Child","Swedish House Mafia"))

[[(1.0,
   "Don't You Worry Child by Swedish House Mafia",
   'https://www.azlyrics.com/lyrics/swedishhousemafia/dontyouworrychild.html'),
  (0.7674418604651163,
   "Don't You Worry Child by Within Temptation",
   'https://www.azlyrics.com/lyrics/withintemptation/dontyouworrychild.html'),
  (0.7532467532467533,
   "Don't You Worry Child by Sam Tsui",
   'https://www.azlyrics.com/lyrics/samtsui/dontyouworrychild.html'),
  (0.7228915662650602,
   "Don't You Worry Child by Madilyn Bailey",
   'https://www.azlyrics.com/lyrics/madilynbailey/dontyouworrychild.html'),
  (0.7160493827160493,
   "Don't You Worry Child by Nicole Cross",
   'https://www.azlyrics.com/lyrics/nicolecross/dontyouworrychild.html')],
 "Don't You Worry Child",
 'Swedish House Mafia']

In [7]:
print(len(scraped_data))

NameError: name 'scraped_data' is not defined

In [8]:
scrape(("Didn't I Blow Your Mind","New Kids On The Block"),debug=False)

[[(0.8888888888888888,
   "Didn't I (Blow Your Mind This Time) by New Kids On The Block",
   'https://www.azlyrics.com/lyrics/newkidsontheblock/didntiblowyourmindthistime.html'),
  (0.42718446601941745,
   'The Session (Longest Posse Cut In History) by The Roots',
   'https://www.azlyrics.com/lyrics/roots/thesessionlongestpossecutinhistory.html'),
  (0.41379310344827586,
   'Dope House Family by South Park Mexican',
   'https://www.azlyrics.com/lyrics/southparkmexican/dopehousefamily.html'),
  (0.4,
   "No One's Iller Than Me by Eminem",
   'https://www.azlyrics.com/lyrics/eminem/noonesillerthanme.html'),
  (0.38961038961038963,
   'One Love (LG Main Mix) by Nas',
   'https://www.azlyrics.com/lyrics/nas/onelovelgmainmix.html'),
  (0.379746835443038,
   'Amen (Intro) by CyHi The Prynce',
   'https://www.azlyrics.com/lyrics/cyhitheprynce/amenintro.html'),
  (0.37777777777777777,
   'Dance With The Devil by Immortal Technique',
   'https://www.azlyrics.com/lyrics/immortaltechnique/dancewi

In [9]:
import multiprocessing

mp_scraped_data = {}
cnt = 0
song_art_list = list(song_art)
song_art_found = []
song_art_missing = []
with multiprocessing.Pool(processes=multiprocessing.cpu_count()*16) as pool:
    for res_ls in pool.imap_unordered(scrape,song_art_list):
        mp_scraped_data[(res_ls[1],res_ls[2])] = res_ls[0]
        if not res_ls[0]:
            song_art_missing.append((res_ls[1],res_ls[2]))
        else:
            song_art_found.append((res_ls[1],res_ls[2]))
        cnt+=1
        if cnt%1000 == 0:
            print("{:4.3f}% done - {} songs 'found'".format(cnt/len(song_art)*100,len(song_art_found)))
pool.close()
pool.terminate()

3.578% done - 733 songs 'found'
7.156% done - 1468 songs 'found'
10.734% done - 2220 songs 'found'
14.312% done - 2942 songs 'found'
17.890% done - 3659 songs 'found'
21.468% done - 4386 songs 'found'
25.046% done - 5100 songs 'found'
28.624% done - 5818 songs 'found'
32.202% done - 6525 songs 'found'
35.779% done - 7248 songs 'found'
39.357% done - 7988 songs 'found'
42.935% done - 8728 songs 'found'
46.513% done - 9451 songs 'found'
50.091% done - 10171 songs 'found'
53.669% done - 10896 songs 'found'
57.247% done - 11613 songs 'found'
60.825% done - 12304 songs 'found'
64.403% done - 13032 songs 'found'
67.981% done - 13740 songs 'found'
71.559% done - 14471 songs 'found'
75.137% done - 15212 songs 'found'
78.715% done - 15938 songs 'found'
82.293% done - 16643 songs 'found'
85.871% done - 17355 songs 'found'
89.449% done - 18092 songs 'found'
93.027% done - 18808 songs 'found'
96.605% done - 19553 songs 'found'


In [10]:
pool.close()
pool.terminate()

27949

In [11]:
print("Search results for {} songs".format(len(song_art_found)))
print("No results for {} songs".format(len(song_art_missing)))

Search results for 20214 songs
No results for 7735 songs


In [12]:
cnt = 0
song_art_list = list(song_art_missing)
song_art_missing = []
with multiprocessing.Pool(processes=multiprocessing.cpu_count()*4) as pool:
    for res_ls in pool.imap_unordered(scrape,song_art_list):
        mp_scraped_data[(res_ls[1],res_ls[2])] = res_ls[0]
        if not res_ls[0]:
            song_art_missing.append((res_ls[1],res_ls[2]))
        else:
            song_art_found.append((res_ls[1],res_ls[2]))
        cnt+=1
        if cnt%1000 == 0:
            print("{:4.3f}% done - {} songs 'found'".format(cnt/len(song_art_list)*100,len(song_art_found)))
pool.close()
pool.terminate()

12.928% done - 20214 songs 'found'
25.856% done - 20214 songs 'found'
38.785% done - 20214 songs 'found'
51.713% done - 20214 songs 'found'
64.641% done - 20214 songs 'found'
77.569% done - 20214 songs 'found'
90.498% done - 20214 songs 'found'


In [13]:
print("Search results for {} songs".format(len(song_art_found)))
print("No results for {} songs".format(len(song_art_missing)))
len(song_art_found) + len(song_art_missing) == len(song_art)

Search results for 20214 songs
No results for 7735 songs


True

In [14]:
for i in range(20):
    print(song_art_missing[i])

('Gilee', 'Sonny Spencer')
('The Way', 'Ariana Grande Featuring Mac Miller')
('Making Memories', 'Frankie Laine')
('East Side Rendezvous', 'Frost')
('Shutters And Boards', 'Jerry Wallace')
("Gimme Gimme Good Lovin'", 'Crazy Elephant')
('Tell Me Something', 'Indecent Obsession')
('Star Love', 'The Playmates')
('Theme From Doctor Detroit', 'Devo')
('Sophisticated Cissy', 'The Meters')
('Image Of A Girl', "Safaris with The Phantom's Band")
('Anytime', 'McAuley Schenker Group')
('Freakit', 'Das EFX')
("The Chokin' Kind", 'Joe Simon')
('L.A. Goodbye', 'The Ides Of March')
('Monkey-Shine', "Bill Black's Combo")
('Special Lady', 'Ray, Goodman & Brown')
('Steal Away', 'Jimmy Hughes')
('Do It To My Mind', 'Johnny Bristol')
('Pay You Back With Interest', "Gary O'")


In [18]:
list(mp_scraped_data.items())[0]

(('Funkdafied', 'Da Brat'),
 [(1.0,
   'Funkdafied by Da Brat',
   'https://www.azlyrics.com/lyrics/dabrat/funkdafied.html'),
  (0.6363636363636364,
   "Mind Blowin' by Da Brat",
   'https://www.azlyrics.com/lyrics/dabrat/mindblowin.html'),
  (0.6363636363636364,
   'Dirty B Side by Da Brat',
   'https://www.azlyrics.com/lyrics/dabrat/dirtybside.html'),
  (0.6086956521739131,
   'World Premiere by Da Brat',
   'https://www.azlyrics.com/lyrics/dabrat/worldpremiere.html'),
  (0.6086956521739131,
   "What 'Chu Like by Da Brat",
   'https://www.azlyrics.com/lyrics/dabrat/whatchulike.html'),
  (0.6086956521739131,
   'Make It Happen by Da Brat',
   'https://www.azlyrics.com/lyrics/dabrat/makeithappen.html'),
  (0.5909090909090909,
   'Give It 2 Ya by Da Brat',
   'https://www.azlyrics.com/lyrics/dabrat/giveit2ya.html'),
  (0.5909090909090909,
   "Fa All Y'all by Da Brat",
   'https://www.azlyrics.com/lyrics/dabrat/faallyall.html'),
  (0.5818181818181818,
   "May Da Funk Be Wit 'Cha by Da Br

In [19]:
from json import dumps,loads

In [21]:
import pickle
print(len(mp_scraped_data))
pickle.dump(mp_scraped_data, open( "../data/pickle/azlyrics_search.pickle", "wb" ) )

27949


In [22]:
sum(int(bool(mp_scraped_data[(s,a)])) for s,a in mp_scraped_data)

20214

### Save the songs with similarity above 0.8

In [41]:
toSave = defaultdict(dict)
for s, a in mp_scraped_data:
    if mp_scraped_data[(s, a)] != []:
        if mp_scraped_data[(s, a)][0][0] > 0.8:
            toSave[a][s] = {"scrapedSongName": mp_scraped_data[(s, a)][0][1], "url": mp_scraped_data[(s, a)][0][2]}

In [62]:
artist_song_pair = set([(j, i) for i in toSave.keys() for j in toSave[i].keys()])
len(artist_song_pair)

16233

In [66]:
missing_artist_song_pair_Set = song_art - artist_song_pair

In [67]:
print(len(missing_artist_song_pair_Set))
assert((len(missing_artist_song_pair_Set) + len(artist_song_pair)) == len(song_art))

11716


In [68]:
missing_artist_song_pair_toSave = defaultdict(set)
for s, a in missing_artist_song_pair_Set:
    missing_artist_song_pair_toSave[a].add(s)

In [71]:
pickle.dump(toSave, open( "../data/pickle/azlyrics_found.pickle", "wb" ) )
pickle.dump(missing_artist_song_pair_toSave, open( "../data/pickle/azlyrics_missing.pickle", "wb" ) )

In [10]:
#map(lambda d: ,BB_data['WeekID'])
def date(s):
    m = re.search(r'^(\d{1,2})\/(\d{1,2})\/(\d{4})$',s)
    return int(m.group(3)),int(m.group(1)),int(m.group(2))
date('12/15/2012')
print(min(map(date,BB_data['WeekID'])))
print(max(map(date,BB_data['WeekID'])))

(1958, 8, 2)
(2018, 12, 29)
